In [12]:
from python_speech_features import mfcc
from python_speech_features import delta
from python_speech_features import logfbank
import scipy.io.wavfile as wav

Nos_Speaker = 10

In [13]:
spk = [wav.read('/home/jugs/PycharmProjects/SpeakerRecognition1.0/audio_preprocessing/audio_sample_data/english' + str(i) + '.wav') for i in range(1,Nos_Speaker+1)]
#(rate, sig) = wav.read('/home/jugs/PycharmProjects/SpeakerRecognition1.0/audio_preprocessing/audio_sample_data/english1.wav') #("/home/jugs/Downloads/LDC93S1.wav")   #LDC93S1.wav
mfcc_feat = [mfcc(spk[i][1], spk[i][0], nfft=1103) for i in range(0,Nos_Speaker)]
d_mfcc_feat = [delta(mfcc_feat[i], 2) for i in range(0,Nos_Speaker)]
fbank_feat = [logfbank(spk[i][1], spk[i][0] , nfft=1103) for i in range(0,Nos_Speaker)]

'''
for i in range(0,9):
    print(fbank_feat[i])
'''

In [14]:
from sklearn.mixture import GaussianMixture
GMM = []
UBM = []

def setGMMUBM(nos_of_component):
    global GMM, UBM
    for i in range(Nos_Speaker):
        GMM.append(GaussianMixture(n_components=nos_of_component, covariance_type='diag'))
        UBM.append(GaussianMixture(n_components=nos_of_component, covariance_type='diag'))

# setGMMUBM(Nos_Speaker)

'''
for i in range(Nos_Speaker):
    print(GMM[i])
'''

'\nfor i in range(Nos_Speaker):\n    print(GMM[i])\n'

In [15]:
# split training and testing data
total_mfcc = []
speaker_label = []
spk_train_size = []
spk_start = []
spk_end = []

for i in range(Nos_Speaker):
    spk_train_size.append(int(0.9 * len(fbank_feat[i])))
    spk_start.append(len(total_mfcc))
    print(i)
    for mfcc in fbank_feat[i][0:spk_train_size[i], :]:
        total_mfcc.append(mfcc)
        speaker_label.append(i)
    spk_end.append(len(total_mfcc))


0
1
2
3
4
5
6
7
8
9


In [22]:
# fitting the GMM and UBM with training data
def fit_model():
    for i in range(Nos_Speaker):
        print("fit starts for {}".format(i))
        GMM[i].fit(fbank_feat[i])
        UBM[i].fit(total_mfcc[:spk_start[i]] + total_mfcc[spk_end[i]:])
        print("fit end for {}".format(i))


In [26]:
def execute():
    fit_model()
    avg_accuracy = 0

    confusion = [[0 for y in range(Nos_Speaker)] for x in range(Nos_Speaker)]

    for i in range(Nos_Speaker):
        for j in range(Nos_Speaker):
            x = GMM[j].score_samples(fbank_feat[i][spk_train_size[i] + 2 : ]) - UBM[j].score_samples(fbank_feat[i][spk_train_size[i] + 2 :])
            for score in x:
                if score > 0:
                    confusion[i][j] += 1

    confusion_diag = [confusion[i][i] for i in range(Nos_Speaker)]

    diag_sum = 0
    for item in confusion_diag:
        diag_sum += item

    remain_sum = 0
    for i in range(Nos_Speaker):
        for j in range(Nos_Speaker):
            if i != j:
                remain_sum += confusion[i][j]

    speaker_accuracy = 0
    for i in range(Nos_Speaker):
        best_guess, _ = max(enumerate(confusion[i]), key=lambda  p:p[1])
        print("for speaker {}, best guess is {}".format(i, best_guess))
        if i == best_guess:
            speaker_accuracy += 1
    speaker_accuracy /= Nos_Speaker

    avg_accuracy = diag_sum/(remain_sum + diag_sum)
    return confusion, avg_accuracy, speaker_accuracy


In [27]:
import numpy as np
if __name__ == "__main__":
    setGMMUBM(300)
    confusion, mfcc_accuracy, spk_accuracy = execute()
    print("Confusion Matrix")
    print(np.matrix(confusion))
    print("MFCC accuracy is {}".format(mfcc_accuracy))
    print("Speaker accuracy is {}".format(spk_accuracy))

fit starts for 0
fit end for 0
fit starts for 1
fit end for 1
fit starts for 2
fit end for 2
fit starts for 3
fit end for 3
fit starts for 4
fit end for 4
fit starts for 5
fit end for 5
fit starts for 6
fit end for 6
fit starts for 7
fit end for 7
fit starts for 8
fit end for 8
fit starts for 9
fit end for 9
for speaker 0, best guess is 0
for speaker 1, best guess is 1
for speaker 2, best guess is 2
for speaker 3, best guess is 3
for speaker 4, best guess is 4
for speaker 5, best guess is 5
for speaker 6, best guess is 6
for speaker 7, best guess is 7
for speaker 8, best guess is 8
for speaker 9, best guess is 9
Confusion Matrix
[[217   0   0   0   0   2   0   0   0   0]
 [  0 206   9   8   2   2   3   0  55   1]
 [  0   4 194   3  33   0   3   2   2  51]
 [  0   6  11 350  45  18  39  47   2   9]
 [  0   3   0  45 289  15  48  48   1   6]
 [  0   1   1  62  71 249  75  40   5   1]
 [  0   6   2  30  39   7 252  42   1   0]
 [  0   0   0  36  29  10  30 200   4   0]
 [  0   0   0   2  

In [32]:
def find_best_param():
    best_no_components = 1
    maxacc = 0
    for i in range(1, 256):
        setGMMUBM(i)
        cm, mfcc_acc, spk_acc = execute()
        print("accuracy for n={} is  {}".format(i, mfcc_acc))
        if mfcc_acc > maxacc:
            maxacc= mfcc_acc
            best_no_component = i
    return best_no_component

a = find_best_param()
print(a)

fit starts for 0
fit end for 0
fit starts for 1
fit end for 1
fit starts for 2
fit end for 2
fit starts for 3
fit end for 3
fit starts for 4
fit end for 4
fit starts for 5
fit end for 5
fit starts for 6
fit end for 6
fit starts for 7
fit end for 7
fit starts for 8
fit end for 8
fit starts for 9
fit end for 9
for speaker 0, best guess is 0
for speaker 1, best guess is 1
for speaker 2, best guess is 2
for speaker 3, best guess is 3
for speaker 4, best guess is 4
for speaker 5, best guess is 5
for speaker 6, best guess is 6
for speaker 7, best guess is 7
for speaker 8, best guess is 8
for speaker 9, best guess is 9
accuracy for n=1 is  0.6949872557349193
fit starts for 0
fit end for 0
fit starts for 1
fit end for 1
fit starts for 2
fit end for 2
fit starts for 3
fit end for 3
fit starts for 4
fit end for 4
fit starts for 5
fit end for 5
fit starts for 6
fit end for 6
fit starts for 7
fit end for 7
fit starts for 8
fit end for 8
fit starts for 9
fit end for 9
for speaker 0, best guess is 0

fit end for 9
for speaker 0, best guess is 0
for speaker 1, best guess is 1
for speaker 2, best guess is 2
for speaker 3, best guess is 3
for speaker 4, best guess is 4
for speaker 5, best guess is 5
for speaker 6, best guess is 6
for speaker 7, best guess is 7
for speaker 8, best guess is 8
for speaker 9, best guess is 9
accuracy for n=13 is  0.703926626540556
fit starts for 0
fit end for 0
fit starts for 1
fit end for 1
fit starts for 2
fit end for 2
fit starts for 3
fit end for 3
fit starts for 4
fit end for 4
fit starts for 5
fit end for 5
fit starts for 6
fit end for 6
fit starts for 7
fit end for 7
fit starts for 8
fit end for 8
fit starts for 9
fit end for 9
for speaker 0, best guess is 0
for speaker 1, best guess is 1
for speaker 2, best guess is 2
for speaker 3, best guess is 3
for speaker 4, best guess is 4
for speaker 5, best guess is 5
for speaker 6, best guess is 6
for speaker 7, best guess is 7
for speaker 8, best guess is 8
for speaker 9, best guess is 9
accuracy for n=1

fit end for 0
fit starts for 1
fit end for 1
fit starts for 2
fit end for 2
fit starts for 3
fit end for 3
fit starts for 4
fit end for 4
fit starts for 5
fit end for 5
fit starts for 6
fit end for 6
fit starts for 7
fit end for 7
fit starts for 8
fit end for 8
fit starts for 9
fit end for 9
for speaker 0, best guess is 0
for speaker 1, best guess is 1
for speaker 2, best guess is 2
for speaker 3, best guess is 3
for speaker 4, best guess is 4
for speaker 5, best guess is 5
for speaker 6, best guess is 6
for speaker 7, best guess is 7
for speaker 8, best guess is 8
for speaker 9, best guess is 9
accuracy for n=26 is  0.7042857142857143
fit starts for 0
fit end for 0
fit starts for 1
fit end for 1
fit starts for 2
fit end for 2
fit starts for 3
fit end for 3
fit starts for 4
fit end for 4
fit starts for 5
fit end for 5
fit starts for 6
fit end for 6
fit starts for 7
fit end for 7
fit starts for 8
fit end for 8
fit starts for 9
fit end for 9
for speaker 0, best guess is 0
for speaker 1, 

fit end for 9
for speaker 0, best guess is 0
for speaker 1, best guess is 1
for speaker 2, best guess is 2
for speaker 3, best guess is 3
for speaker 4, best guess is 4
for speaker 5, best guess is 5
for speaker 6, best guess is 6
for speaker 7, best guess is 7
for speaker 8, best guess is 8
for speaker 9, best guess is 9
accuracy for n=38 is  0.7075064710957722
fit starts for 0
fit end for 0
fit starts for 1
fit end for 1
fit starts for 2
fit end for 2
fit starts for 3
fit end for 3
fit starts for 4
fit end for 4
fit starts for 5
fit end for 5
fit starts for 6
fit end for 6
fit starts for 7
fit end for 7
fit starts for 8
fit end for 8
fit starts for 9
fit end for 9
for speaker 0, best guess is 0
for speaker 1, best guess is 1
for speaker 2, best guess is 2
for speaker 3, best guess is 3
for speaker 4, best guess is 4
for speaker 5, best guess is 5
for speaker 6, best guess is 6
for speaker 7, best guess is 7
for speaker 8, best guess is 8
for speaker 9, best guess is 9
accuracy for n=

KeyboardInterrupt: 